# Lesson 2: LLM Providers & Models

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harshit-vibes/lyzr-adk-demo/blob/master/notebooks/02_providers_and_models.ipynb)


🟢 Beginner · ⏱ 20 min

lyzr-adk lets you switch between AI providers — OpenAI, Anthropic, Google, Groq, Perplexity, and AWS Bedrock — with a single string. This lesson shows you how to create agents backed by different LLMs and swap providers on an existing agent without recreating it.

**What you'll learn:**
- Understand the `provider/model-name` string format
- Create agents with different LLM providers
- Compare responses from two providers side by side
- Switch providers on an existing agent using `agent.update()`

## Prerequisites

- **Lesson 1 completed** — you should already know how to create an agent and call `agent.run()`
- `LYZR_API_KEY` environment variable set (or ready to paste your key below)
- API keys for any additional providers you want to test (e.g. `OPENAI_API_KEY`, `ANTHROPIC_API_KEY`) — the Lyzr platform routes calls through your own provider keys

In [ ]:
!pip install lyzr-adk -q

In [ ]:
import os
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Studio ready!")

## The Provider String Format

Every agent in lyzr-adk is backed by a specific LLM. You specify the LLM using a **provider string** in the format:

```
"provider/model-name"
```

The part before the `/` is the **provider** (the company or platform hosting the model). The part after is the **model name** exactly as that provider refers to it.

Here are all the providers supported in lyzr-adk:

| Provider | String | Notes |
|----------|--------|-------|
| OpenAI | `openai/gpt-4o` | Default, versatile, great all-rounder |
| Anthropic | `anthropic/claude-sonnet-4-5` | Strong reasoning and instruction-following |
| Google | `google/gemini-2.0-flash` | Fast, multimodal (text + images) |
| Groq | `groq/llama-3.3-70b` | Very fast inference via Groq hardware |
| Perplexity | `perplexity/sonar-pro` | Web-grounded — can cite live sources |
| AWS Bedrock | `bedrock/amazon.nova-pro-v1:0` | Enterprise deployments on AWS |

> **Note:** You only need an API key for the provider you are actually calling. If you only have an OpenAI key, you can only use `openai/...` models.

In [ ]:
# All supported providers in lyzr-adk
PROVIDERS = {
    "OpenAI":      "openai/gpt-4o",
    "Anthropic":   "anthropic/claude-sonnet-4-5",
    "Google":      "google/gemini-2.0-flash",
    "Groq":        "groq/llama-3.3-70b",
    "Perplexity":  "perplexity/sonar-pro",
}

print("Supported provider strings:")
for name, provider_string in PROVIDERS.items():
    print(f"  {name:12} → {provider_string}")

## Creating Agents with Different Providers

The code to create an agent is identical regardless of provider — the only thing that changes is the `provider` argument. This makes it easy to experiment with different LLMs without restructuring your code.

In the example below, we create two agents with the same role, goal, and instructions, but backed by different models. This is the cleanest way to set up a fair comparison.

In [ ]:
# Create two agents with different providers
openai_agent = studio.create_agent(
    name="OpenAI Agent",
    provider="openai/gpt-4o",
    role="Helpful assistant",
    goal="Answer questions concisely",
    instructions="Keep responses to 1-2 sentences."
)

google_agent = studio.create_agent(
    name="Google Agent",
    provider="google/gemini-2.0-flash",
    role="Helpful assistant",
    goal="Answer questions concisely",
    instructions="Keep responses to 1-2 sentences."
)

print(f"OpenAI agent:  {openai_agent.id}")
print(f"Google agent:  {google_agent.id}")

## Comparing Responses

Because each model has been trained differently, they often produce distinct phrasings, levels of detail, and stylistic choices — even for simple factual questions. Asking both agents the same question and printing the results side by side is the quickest way to feel those differences.

In [ ]:
question = "In one sentence, what is machine learning?"

openai_response = openai_agent.run(question)
google_response = google_agent.run(question)

print(f"OpenAI:  {openai_response.response}")
print(f"")
print(f"Google:  {google_response.response}")

## Switching Provider on an Existing Agent

You don't need to create a brand-new agent to change the underlying model. Call `agent.update(provider=...)` to swap the LLM in place — the agent ID, role, goal, and instructions all stay the same.

This is useful when you want to:
- A/B test different models in production without changing agent IDs
- Upgrade to a newer model version without reconfiguring everything
- Fall back to a cheaper or faster model when latency matters

In [ ]:
print(f"Before update: openai_agent uses OpenAI")
openai_agent.update(provider="anthropic/claude-sonnet-4-5")
print(f"After update: openai_agent now uses Anthropic")

response = openai_agent.run("What is 2 + 2? One sentence only.")
print(f"Response: {response.response}")

## Common Mistake: Invalid Provider String Format

The provider string **must** follow the `provider/model-name` pattern exactly. A common mistake is passing just the model name without the provider prefix — for example, `"gpt-4o"` instead of `"openai/gpt-4o"`.

lyzr-adk validates the format when you create or update an agent. If the string is invalid or references a model that doesn't exist, you'll get an error immediately rather than at inference time.

In [ ]:
# ❌ Common mistake: wrong format or non-existent model
try:
    bad_agent = studio.create_agent(
        name="Bad Agent",
        provider="gpt-4o",       # ❌ Missing "openai/" prefix
        role="Test", goal="Test", instructions="Test"
    )
except Exception as e:
    print(f"Error (expected): {e}")

# ✅ Correct format always: "provider/model-name"
print("\n✅ Correct format: 'openai/gpt-4o', 'google/gemini-2.0-flash', etc.")

## Exercise

Create two agents backed by **different providers** from the `PROVIDERS` dict above. Ask both the same question about space and print their responses side by side.

**Acceptance criteria:**
- Both agents use a different entry from `PROVIDERS`
- Both agents have the same `role`, `goal`, and `instructions`
- You print each response labeled with the provider name

In [ ]:
# TODO: Choose two different providers from PROVIDERS dict above
agent_a = studio.create_agent(
    name=...,
    provider=...,   # e.g. PROVIDERS["OpenAI"]
    role="Trivia expert",
    goal="Share interesting facts",
    instructions="Give exactly one fun fact in one sentence."
)

agent_b = studio.create_agent(
    name=...,
    provider=...,   # Pick a different provider
    role="Trivia expert",
    goal="Share interesting facts",
    instructions="Give exactly one fun fact in one sentence."
)

# TODO: Ask both the same question and print side by side
question = "Give me a fun fact about space."
# ...

## Summary

| Provider | String | Best for |
|----------|--------|----------|
| OpenAI | `openai/gpt-4o` | General-purpose tasks, code, reasoning |
| Anthropic | `anthropic/claude-sonnet-4-5` | Long documents, nuanced instruction-following |
| Google | `google/gemini-2.0-flash` | Speed, multimodal inputs |
| Groq | `groq/llama-3.3-70b` | Ultra-low latency inference |
| Perplexity | `perplexity/sonar-pro` | Questions requiring up-to-date web data |
| AWS Bedrock | `bedrock/amazon.nova-pro-v1:0` | Enterprise / AWS-native deployments |

**Key takeaways:**
- Provider strings always follow `"provider/model-name"` — no exceptions
- You only need an API key for the provider you actually call
- Create agents with different providers: `studio.create_agent(..., provider="openai/gpt-4o")`
- Switch provider on an existing agent: `agent.update(provider="google/gemini-2.0-flash")`
- Switching provider does **not** change the agent ID, role, goal, or instructions

## Next Steps

➡️ **Lesson 3: Agent Lifecycle**

Now that you can create agents with any provider, learn to manage them over time — list all your agents, retrieve one by ID, clone an existing agent into a new one, and delete agents you no longer need.